In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from scraping_helper_functions import cook_chess_soup
from scraping_helper_functions import get_chess_data

In [2]:
# This is an extremely important cell, very very necessary
def lentil(x):
    return len(x)

In [3]:
# URL = "https://www.chess.com/games/search?opening=&openingId=&p1=Magnus+Carlsen&p2=&mr=&lsty=1&year=&lstMoves=1&moves=&fen=&sort=page=1"
# page = requests.get(URL)

# soup = BeautifulSoup(page._content, "html.parser")

Scrape Chess.com for all of Magnus's otb chess games

In [4]:
# broth = soup.html.body.find('div', class_ = "base-layout").find('div', class_ = "base-container").main
# veggies = broth.find('div', class_ = "layout-column-one").find("div", class_ = "v5-section").find('div', class_ = "v5-section-content-wide").find_all('div', class_ = "post-preview-list-component")[2]
# noodles = veggies.find('div', class_ = 'master-games-table-responsive').table.tbody
# meats = noodles.find_all('tr')

Now we have a list of table entries in html that contains data about each of Magnus's games (on this page at least)
So, lets loop through this list and extract the important data into a dataframe 

Loop through the different pages and grab all the list elements containing data about Magnus's otb games

In [5]:
html_data = []

# Right now this only scrapes games up until 2015 (the first 60 pages)
for num in range(0, 60):
    URL = f"https://www.chess.com/games/search?opening=&openingId=&p1=Magnus%20Carlsen&p2=&mr=&lsty=1&year=&lstMoves=1&moves=&fen=&sort=page%3D3&page={num}"
    html_data.extend(cook_chess_soup(URL))

In [33]:
lentil(html_data)

1470

In [18]:
white_player_list = []
white_player_rating_list = []
black_player_list = []
black_player_rating_list = []
result_list = []
game_length_list = []
year_list = []
opening_name_list = []
opening_list = []

for chunk in html_data:
    data = get_chess_data(chunk)

    white_player_list.append(data[0])
    white_player_rating_list.append(data[1])
    black_player_list.append(data[2])
    black_player_rating_list.append(data[3])
    result_list.append(data[4])
    game_length_list.append(data[5])
    year_list.append(data[6])
    opening_name_list.append(data[7])
    opening_list.append(data[8])

White player rating nan
Black player rating nan
White player rating nan
Black player rating nan
White player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan


Now that we have all the lists of data, build the dataframe

In [24]:
games_df = pd.DataFrame()

#There is probably a better way to do this but whatever
games_df['white_player'] = white_player_list
games_df['white_player_rating'] = white_player_rating_list
games_df['black_player'] = black_player_list
games_df['black_player_rating'] = black_player_rating_list
games_df['result'] = result_list
games_df['length'] = game_length_list
games_df['year'] = year_list
games_df['opening_name'] = opening_name_list
games_df['opening_moves'] = opening_list

In [25]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   white_player         1470 non-null   object
 1   white_player_rating  1467 non-null   object
 2   black_player         1470 non-null   object
 3   black_player_rating  1456 non-null   object
 4   result               1470 non-null   object
 5   length               1470 non-null   object
 6   year                 1470 non-null   object
 7   opening_name         1470 non-null   object
 8   opening_moves        1470 non-null   object
dtypes: object(9)
memory usage: 103.5+ KB


str

Ok, we have the data. Lets clean it.

In [32]:
#Lets do the easy things first: cleaning the ratings, year, result, length, and opening
games_df['result'] = games_df['result'].map(lambda x: x.strip())
games_df['length'] = games_df['length'].map(lambda x: x.strip())
games_df['opening_moves'] = games_df['opening_moves'].map(lambda x: x.strip())
games_df['opening_name'] = games_df['opening_name'].map(lambda x: x.strip())

games_df

AttributeError: 'Series' object has no attribute 'info'

In [40]:
def rating_to_int(rating):
    if type(rating) != float:
        return int(rating[1:5])
    else:
        return rating

In [41]:
games_df['white_player_rating'] = games_df['white_player_rating'].map(rating_to_int)
games_df['black_player_rating'] = games_df['black_player_rating'].map(rating_to_int)

In [44]:
games_df

,white_player,white_player_rating,black_player,black_player_rating,result,length,year,opening_name,opening_moves
0,Magnus Carlsen,2881.0,Wesley So,2741.0,½-½,29,2021,"Ruy López Opening: Berlin, Rio Gambit Accepted...",1. e4 e5 2. Nf3 Nc6 3. Bb5 Nf6
1,Wesley So,2816.0,Magnus Carlsen,2886.0,0-1,36,2021,"Sicilian Defense: Canal Attack, 3...Nd7 4.O-O ...",1. e4 c5 2. Nf3 d6 3. Bb5+ Nd7
2,Hikaru Nakamura,2736.0,Magnus Carlsen,2847.0,½-½,38,2021,"Ruy López Opening: Berlin Defense, 4.d3 Bc5",1. e4 e5 2. Nf3 Nc6 3. Bb5 Nf6
3,Santosh Gujrathi Vidit,2726.0,Magnus Carlsen,2847.0,½-½,75,2021,"Nimzo-Indian Defense: Classical, Noa Variation...",1. d4 Nf6 2. c4 e6 3. Nc3 Bb4
4,Etienne Bacrot,2678.0,Magnus Carlsen,2847.0,0-1,44,2021,"Ruy López Opening: Morphy Defense, Anti-Marsha...",1. e4 e5 2. Nf3 Nc6 3. Bb5 a6
...,...,...,...,...,...,...,...,...,...
1465,Magnus Carlsen,2876.0,Veselin Topalov,2798.0,0-1,60,2015,Semi-Slav Defense,1. d4 d5 2. c4 c6 3. Nf3 Nf6
1466,Magnus Carlsen,2862.0,Ivan Saric,2666.0,½-½,49,2015,Sicilian Defense: Old Sicilian Variation,1. e4 c5 2. Nf3 Nc6 3. Nc3 Nf6
1467,Magnus Carlsen,2853.0,Maxime Vachier-Lagrave,2731.0,1-0,43,2015,"English Opening: Anglo-Indian, King's Indian D...",1. Nf3 Nf6 2. c4 g6 3. g3 Bg7
1468,Magnus Carlsen,2853.0,Alexander Grischuk,2771.0,0-1,66,2015,"Sicilian Defense: Open, Najdorf, Opočenský Var...",1. e4 c5 2. Nf3 d6 3. d4 cxd4
